In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

try:
    model = tf.keras.models.load_model('apple_orange_classifier.h5')
    print("Model loaded successfully!")
except:
    print("Error loading model. Make sure 'apple_orange_classifier.h5' exists in the current directory.")

class_names = ['Apple', 'Orange']

def predict_image(image):
    try:
        if image is None:
            return "Please upload an image", None
        
        image_resized = image.resize((32, 32))
        img_array = np.array(image_resized)
        img_array = img_array.astype('float32') / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        prediction = model.predict(img_array, verbose=0)
        probability = float(prediction[0][0])
        
        if probability > 0.5:
            predicted_class = "Orange"
            confidence = probability * 100
        else:
            predicted_class = "Apple"
            confidence = (1 - probability) * 100
        
        result = f"Prediction: {predicted_class}\nConfidence: {confidence:.2f}%"
        
        return result, image_resized
        
    except Exception as e:
        return f"Error processing image: {str(e)}", None

def predict_with_details(image):
    try:
        if image is None:
            return "Please upload an image", None, None
        
        image_resized = image.resize((32, 32))
        img_array = np.array(image_resized)
        img_array = img_array.astype('float32') / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        prediction = model.predict(img_array, verbose=0)
        probability = float(prediction[0][0])
        
        apple_prob = (1 - probability) * 100
        orange_prob = probability * 100
        
        result = f"""
        Apple: {apple_prob:.2f}%
        Orange: {orange_prob:.2f}%
        
        Final Prediction: {'Orange' if probability > 0.5 else 'Apple'}
        """
        
        fig, ax = plt.subplots(figsize=(8, 6))
        classes = ['Apple', 'Orange']
        probabilities = [apple_prob, orange_prob]
        colors = ['red', 'orange']
        
        bars = ax.bar(classes, probabilities, color=colors, alpha=0.7)
        ax.set_ylabel('Probability (%)')
        ax.set_title('Classification Probabilities')
        ax.set_ylim(0, 100)
        
        for bar, prob in zip(bars, probabilities):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                   f'{prob:.1f}%', ha='center', va='bottom')
        
        plt.tight_layout()
        
        return result, image_resized, fig
        
    except Exception as e:
        return f"Error: {str(e)}", None, None

with gr.Blocks(title="Apple vs Orange Classifier", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Apple vs Orange Classifier")
    gr.Markdown("Upload an image and the AI will classify it as either an Apple or Orange!")
    
    with gr.Tab("Simple Prediction"):
        with gr.Row():
            with gr.Column():
                input_image1 = gr.Image(type="pil", label="Upload Image")
                predict_btn1 = gr.Button("Classify Image", variant="primary")
            
            with gr.Column():
                output_text1 = gr.Textbox(label="Prediction Result", lines=3)
                output_image1 = gr.Image(label="Processed Image (32x32)")
        
        predict_btn1.click(
            fn=predict_image,
            inputs=input_image1,
            outputs=[output_text1, output_image1]
        )
    
    with gr.Tab("Detailed Analysis"):
        with gr.Row():
            with gr.Column(scale=1):
                input_image2 = gr.Image(type="pil", label="Upload Image")
                predict_btn2 = gr.Button("Analyze Image", variant="primary")
                output_text2 = gr.Textbox(label="Detailed Results", lines=6)
            
            with gr.Column(scale=1):
                output_image2 = gr.Image(label="Processed Image (32x32)")
                output_plot = gr.Plot(label="Probability Distribution")
        
        predict_btn2.click(
            fn=predict_with_details,
            inputs=input_image2,
            outputs=[output_text2, output_image2, output_plot]
        )
    
    gr.Markdown("## Example Images")
    gr.Markdown("Try uploading images of apples or oranges to test the classifier!")
    
    input_image1.change(
        fn=predict_image,
        inputs=input_image1,
        outputs=[output_text1, output_image1]
    )

if __name__ == "__main__":
    demo.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860
    )
